# 1.0 Libraries & Set up

## In this section we import necessary libraries and apply plotting settings to the entire notebook

In [ ]:
import os
os.chdir('../') # insures use of models folder as python module 
# Any figures saved or data pickled in the current working 
# directly will be saved up one folder from here.
import pybamm
import models
import numpy as np
import pickle
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['font.size'] =  18.0

In [ ]:
def run_simulation(sim,time_in_seconds = 43200):
    time = 0
    while time < time_in_seconds:
        try:
            sim.step(5)
            time+= 5
        except:
            try: 
                sim.step(.1)
                time += .1
            except:
                try:
                    sim.step(0.0001)
                    time += 0.0001
                except:
                    break
                    
    return sim

# 1.1 Single Layer Cell held at 30 degrees C

## 1.1.1 Initial conditions and basic parameters

### Parameter values and initial values as discussed in Cornish & Marinescu (2021) regarding equations (8), (9a), and (9b).

In [ ]:
# Experimental cell capacity
cap = 0.211

# Parameters taken from Marinescu et al. (2016)
Ms = 32
F = 9.649e4
R = 8.3145
T_30 = 303.15
ar = 0.960
nu = 0.0114

f_h = ((4 ** 2) * Ms * nu / 8)**(1/4)
f_m = ((2 ** 2) * Ms * nu / 4)**(1/2)
f_l = ((1 ** 2) * Ms * nu / 2)**(1/2)

# Current at 0.2C rate
I = -cap*0.1

# New standard potentials
EL0 = 1.9
EM0 = 2.41
EH0 = 2.43

# New exchange current constants
ih0 = 5
im0 = 5
il0 = 5

# Saturation mass constant
S_star_30 = 1e-6

# dissolution rate
kd_30 = 5000

# shuttle rate
ks_30 = 1e-5

# Initial value of Precipitated sulfur at start of charge
Sp_initial_cha_30 = .43

# Initial value of S1 anion at start of charge
S1_initial_cha_30 = 1e-3

# Initial value of voltage at start of charge
V_initial_cha = 2.0

# Function for which root yields i_L = I
def S2_func(S1):
    constant = np.exp((2*F/(R*T_30))*(V_initial_cha - (2*R*T_30/F)*np.arcsinh(I/(-2*ar*il0)) - EL0 ))/(f_l**2)
    return constant*(S1**2)

# Function from equation (9b) in Cornish & Marinescu (2021)
def S4_func(S2):
    constant = np.exp((2*F/(R*T_30))*(V_initial_cha  - EM0 ))/(f_m**2)
    return constant*(S2**2)

# Function from equation (9a) in Cornish & Marinescu (2021)
def S8_func(S4):
    constant = np.exp((4*F/(R*T_30))*(V_initial_cha - EH0 ))/(f_h**4)
    return constant*(S4**2)

# Initial charge value of S2
S2_initial_cha_30 = S2_func(S1_initial_cha_30)

# Initial charge value of S2
S4_initial_cha_30 = S4_func(S2_initial_cha_30)

# Initial charge value of S2
S8_initial_cha_30 = S8_func(S4_initial_cha_30)

# Initial high plateau potential 
EH_30 = EH0 + (R*T_30)*np.log(f_h*(S8_initial_cha_30**(1/4))/(S4_initial_cha_30**(1/2)))/(F)

# Initial high plateau over-potential
eta_H_30 = V_initial_cha - EH_30

# Initial high plateau current
iH_30 = -2*ih0*ar*np.sinh(F*eta_H_30/(2*R*T_30))

# Initial middle plateau potential
EM_30 = EM0 + (R*T_30)*np.log(f_m*(S4_initial_cha_30**(1/2))/S2_initial_cha_30)/(F)

# Initial middle plateau over-potentia
eta_M_30 = V_initial_cha - EM_30

# Initial middle plateau current
iM_30 = -2*im0*ar*np.sinh(F*eta_M_30/(2*R*T_30))

# Initial low plateau potential
EL_30 = EL0 + (R*T_30/F)*np.log(f_l*(S2_initial_cha_30**(1/2))/S1_initial_cha_30)

# Initial low plateau over-potentia
eta_L_30 = V_initial_cha - EL_30

# Initial low plateau current
iL_30 = -2*il0*ar*np.sinh(F*eta_L_30/(2*R*T_30))

print('The initial charge value of precipitated sulfur is {} g'.format(Sp_initial_cha_30))
print('The initial charge value of S1 anion is {} g'.format(S1_initial_cha_30))
print('The initial charge value of S2 anion is {} g'.format(S2_initial_cha_30))
print('The initial charge value of S4 anion is {} g'.format(S4_initial_cha_30))
print('The initial charge value of S8 anion is {} g'.format(S8_initial_cha_30))
print('The initial charge value of voltage is {} V'.format(V_initial_cha))
print('The initial charge value of the high plateau potential is {} V'.format(EH_30))
print('The initial charge value of the high plateau over-potential is {} V'.format(eta_H_30))
print('The initial charge value of the high plateau current is {} A'.format(iH_30))
print('The initial charge value of the middle plateau potential is {} V'.format(EM_30))
print('The initial charge value of the middle plateau over-potential is {} V'.format(eta_M_30))
print('The initial charge value of the middle plateau current is {} A'.format(iM_30))
print('The initial charge value of the low plateau potential is {} V'.format(EL_30))
print('The initial charge value of the low plateau over-potential is {} V'.format(eta_L_30))
print('The initial charge value of the low plateau current is {} A'.format(iL_30))

## 1.1.2 Fit 30 Degree Cell Charge Voltage 

In [ ]:
# Instantiate the model
model_30 = models.Cornish2021()


# Obtain model parameter dictionary
param_30 = model_30.default_parameter_values

# Update the parameters for charge
param_30.update(
    {
        "Current function [A]": I,
        "Initial Condition for Terminal Voltage [V]": V_initial_cha,
        "Initial Condition for S8 ion [g]" : S8_initial_cha_30,
        "Initial Condition for S4 ion [g]" : S4_initial_cha_30,
        "Initial Condition for S2 ion [g]" : S2_initial_cha_30,
        "Initial Condition for S ion [g]" : S1_initial_cha_30,
        "Initial Condition for Precipitated Sulfur [g]" : Sp_initial_cha_30,
        "Lower voltage cut-off [V]" : 1.2,
        "Upper voltage cut-off [V]" : 2.5,
        "Standard Potential L [V]" : EL0,
        "Standard Potential M [V]" : EM0,
        "Standard Potential H [V]" : EH0,
        "Exchange current density L [A.m-2]": il0,
        "Exchange current density M [A.m-2]": im0,
        "Exchange current density H [A.m-2]": ih0,
        "Active reaction area per cell [m2]":ar,
        "S saturation mass [g]": S_star_30,
        'Shuttle rate coefficient during charge [s-1]': ks_30,
        'Dissolution rate [s-1]': kd_30,
        'Reference Temperature [K]': T_30
        }
)

# Instantiate simulation object 
sim_30_cha1 = pybamm.Simulation(
    model_30,
    parameter_values = param_30,
    solver=pybamm.CasadiSolver(
        atol=1e-6,
        rtol=1e-6,
        dt_max = .01,
        extra_options_setup={"max_step_size": 0.1}, 
        max_step_decrease_count = 15,
    ),
)

# Define the time cut-off for charging in second
time_in_seconds = 12*int(3600/0.1)

# Simulate the model 
#sim_30_cha1.solve([0,time_in_seconds]);


sim_30_cha1 = run_simulation(sim_30_cha1)

In [ ]:
# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

#plt.plot(charge_data['30']['capacity'],charge_data['30']['internal voltage'], linestyle='--')

index_30 = len(sim_30_cha1.solution.t[sim_30_cha1.solution.t*0.0211/3600 < charge_data['30']['capacity'][-1]])

capacity = abs(I)*sim_30_cha1.solution.t[:index_30]/3600
voltage = sim_30_cha1.solution['Terminal voltage [V]'].entries[:index_30]


plt.plot(capacity, voltage)
plt.title("Charge Voltage Fit for $30^\circ$ Cell")
plt.legend(['Experiment 0.1C', 'Simulation 0.1C'], loc = 'lower right', frameon=False)
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

## 1.1.3 Fit the Charge Resistance

In [ ]:
capacity_30 = .0211*sim_30_cha1.solution.t/3600
S4 = sim_30_cha1.solution['S4 [g]'].entries
S2 = sim_30_cha1.solution['S2 [g]'].entries
S1 = sim_30_cha1.solution['S [g]'].entries
Li_30 = S1 + 0.5*S2 + 0.25*S4 

filename = 'Cornish & Marinescu (2021)/res_data'
infile = open(filename,'rb')
res_data = pickle.load(infile)
infile.close()

res_discharge_data = res_data['discharge']
res_charge_data = res_data['charge']
plt.plot(res_charge_data['30'][0],res_charge_data['30'][1],linestyle='--')

gamma_30 = 1.25
beta_30 = max(Li_30)*gamma_30
alpha_30 = (gamma_30-1)*max(res_charge_data['30'][1])*max(Li_30)
R_30 = alpha_30/(beta_30 - Li_30)

print(alpha_30)
print(beta_30)

plt.plot(capacity_30,R_30)

plt.title("Charge Resistance Fit for $30^\circ$ Cell")
plt.legend(['Experiment 0.1C', 'Simulation 0.1C'])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Resistance [$\Omega$]');

## 1.1.4 Fit The Discharge Voltage Curve

In [ ]:
loc = index_30
S1_initial_30_dis1 = sim_30_cha1.solution['S [g]'].entries[loc]
S2_initial_30_dis1 = sim_30_cha1.solution['S2 [g]'].entries[loc]
S4_initial_30_dis1 = sim_30_cha1.solution['S4 [g]'].entries[loc]
S8_initial_30_dis1 = sim_30_cha1.solution['S8 [g]'].entries[loc]
Sp_initial_30_dis1 = sim_30_cha1.solution['Precipitated Sulfur [g]'].entries[loc]
V_initial_30_dis1 = sim_30_cha1.solution['Terminal voltage [V]'].entries[loc]
print(S8_initial_30_dis1)
print(S4_initial_30_dis1)
print(S2_initial_30_dis1)
print(S1_initial_30_dis1)
print(Sp_initial_30_dis1)
print(V_initial_30_dis1)

In [ ]:
model_30_dis1 = models.Cornish2021()

param_30_dis1 = model_30_dis1.default_parameter_values

# dissolution rate
kp_30 = 1
param_30_dis1.update(
    {
        "Current function [A]": 2*.0211,
        "Initial Condition for Terminal Voltage [V]": V_initial_30_dis1,
        "Initial Condition for S8 ion [g]" : S8_initial_30_dis1,
        "Initial Condition for S4 ion [g]" : S4_initial_30_dis1,
        "Initial Condition for S2 ion [g]" : S2_initial_30_dis1,
        "Initial Condition for S ion [g]" : S1_initial_30_dis1,
        "Initial Condition for Precipitated Sulfur [g]" : Sp_initial_30_dis1,
        "Lower voltage cut-off [V]" : 1.2,
        "Upper voltage cut-off [V]" : 2.5,
        "Standard Potential L [V]" : EL0,
        "Standard Potential M [V]" : EM0,
        "Standard Potential H [V]" : EH0,
        "Exchange current density L [A.m-2]": il0,
        "Exchange current density M [A.m-2]": im0,
        "Exchange current density H [A.m-2]": ih0,
        "Active reaction area per cell [m2]":ar,
        "S saturation mass [g]": S_star_30,
        'Precipitation rate [s-1]': kp_30,
        'Reference Temperature [K]': T_30
        }
)


sim_30_dis1 = pybamm.Simulation(
    model_30_dis1,
    parameter_values = param_30_dis1,
    solver=pybamm.CasadiSolver(
        atol=1e-8,
        rtol=1e-6,
        dt_max = .01,
        #extra_options_setup={"max_step_size": 0.1}, 
        max_step_decrease_count = 15,
    ),
)

sim_30_dis1 = run_simulation(sim_30_dis1, time_in_seconds = 18220)


In [ ]:
plt.plot(sim_30_dis1.solution.t*0.0422/3600, sim_30_dis1.solution['Terminal voltage [V]'].entries)

# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

plt.plot(discharge_data['30']['capacity'],discharge_data['30']['internal voltage'], linestyle='--')

plt.title("Disharge Voltage Fit for $30^\circ$ Cell")
plt.legend(["Simulation 0.2C","Experiment 0.2C" ])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

# 1.2 The 40 degree Cell

In [ ]:
# Experimental cell capacity
cap = 0.211

# Parameters taken from Marinescu et al. (2016)
Ms = 32
F = 9.649e4
R = 8.3145
T_40 = 303.15 + 10
ar = 0.960
nu = 0.0114

f_h = ((4 ** 2) * Ms * nu / 8)**(1/4)
f_m = ((2 ** 2) * Ms * nu / 4)**(1/2)
f_l = ((1 ** 2) * Ms * nu / 2)**(1/2)

# Current at 0.2C rate
I = -cap*0.1

# New standard potentials
EL0 = 1.9
EM0 = 2.41
EH0 = 2.43

# Exchange current constants
ih0 = 5
im0 = 5
il0 = 5

# Saturation mass constant
S_star_40 = S_star_30#2.5e-6

# dissolution rate
kd_40 = kd_30#5000# 4400

# shuttle rate
ks_40 = 1e-3

# Initial value of S1 anion at start of charge
S1_initial_cha_40 = S1_initial_cha_30*10#/50# 5e-4

# Initial value of voltage at start of charge
V_initial_cha = 2.0

# Function for which root yields i_L = I
def S2_func(S1):
    constant = np.exp((2*F/(R*T_40))*(V_initial_cha - (2*R*T_40/F)*np.arcsinh(I/(-2*ar*il0)) - EL0 ))/(f_l**2)
    return constant*(S1**2)

# Function from equation (9b) in Cornish & Marinescu (2021)
def S4_func(S2):
    constant = np.exp((2*F/(R*T_40))*(V_initial_cha  - EM0 ))/(f_m**2)
    return constant*(S2**2)

# Function from equation (9a) in Cornish & Marinescu (2021)
def S8_func(S4):
    constant = np.exp((4*F/(R*T_40))*(V_initial_cha - EH0 ))/(f_h**4)
    return constant*(S4**2)

# Initial charge value of S2
S2_initial_cha_40 = S2_func(S1_initial_cha_40)

# Initial charge value of S2
S4_initial_cha_40 = S4_func(S2_initial_cha_40)

# Initial charge value of S2
S8_initial_cha_40 = S8_func(S4_initial_cha_40)

# Initial value of Precipitated sulfur at start of charge
# mass conservation across cells
Sp_initial_cha_40 = (S8_initial_cha_30 + 
                     S4_initial_cha_30 + 
                     S2_initial_cha_30 + 
                     S1_initial_cha_30 + 
                     Sp_initial_cha_30) - (S8_initial_cha_40 +
                                          S4_initial_cha_40 + 
                                          S2_initial_cha_40 + 
                                          S1_initial_cha_40)

# Initial high plateau potential 
EH_40 = EH0 + (R*T_40)*np.log(f_h*(S8_initial_cha_40**(1/4))/(S4_initial_cha_40**(1/2)))/(F)

# Initial high plateau over-potential
eta_H_40 = V_initial_cha - EH_40

# Initial high plateau current
iH_40 = -2*ih0*ar*np.sinh(F*eta_H_40/(2*R*T_40))

# Initial middle plateau potential
EM_40 = EM0 + (R*T_40)*np.log(f_m*(S4_initial_cha_40**(1/2))/S2_initial_cha_40)/(F)

# Initial middle plateau over-potentia
eta_M_40 = V_initial_cha - EM_40

# Initial middle plateau current
iM_40 = -2*im0*ar*np.sinh(F*eta_M_40/(2*R*T_40))

# Initial low plateau potential
EL_40 = EL0 + (R*T_40/F)*np.log(f_l*(S2_initial_cha_40**(1/2))/S1_initial_cha_40)

# Initial low plateau over-potentia
eta_L_40 = V_initial_cha - EL_40

# Initial low plateau current
iL_40 = -2*il0*ar*np.sinh(F*eta_L_40/(2*R*T_40))

print('The initial charge value of precipitated sulfur is {} g'.format(Sp_initial_cha_40))
print('The initial charge value of S1 anion is {} g'.format(S1_initial_cha_40))
print('The initial charge value of S2 anion is {} g'.format(S2_initial_cha_40))
print('The initial charge value of S4 anion is {} g'.format(S4_initial_cha_40))
print('The initial charge value of S8 anion is {} g'.format(S8_initial_cha_40))
print('The initial charge value of voltage is {} V'.format(V_initial_cha))
print('The initial charge value of the high plateau potential is {} V'.format(EH_40))
print('The initial charge value of the high plateau over-potential is {} V'.format(eta_H_40))
print('The initial charge value of the high plateau current is {} A'.format(iH_40))
print('The initial charge value of the middle plateau potential is {} V'.format(EM_40))
print('The initial charge value of the middle plateau over-potential is {} V'.format(eta_M_40))
print('The initial charge value of the middle plateau current is {} A'.format(iM_40))
print('The initial charge value of the low plateau potential is {} V'.format(EL_40))
print('The initial charge value of the low plateau over-potential is {} V'.format(eta_L_40))
print('The initial charge value of the low plateau current is {} A'.format(iL_40))

In [ ]:
# Instantiate the model
model_40 = models.Cornish2021()

# Obtain model parameter dictionary
param_40 = model_40.default_parameter_values

# Update the parameters for charge
param_40.update(
    {
        "Current function [A]": I,
        "Initial Condition for Terminal Voltage [V]": V_initial_cha,
        "Initial Condition for S8 ion [g]" : S8_initial_cha_40,
        "Initial Condition for S4 ion [g]" : S4_initial_cha_40,
        "Initial Condition for S2 ion [g]" : S2_initial_cha_40,
        "Initial Condition for S ion [g]" : S1_initial_cha_40,
        "Initial Condition for Precipitated Sulfur [g]" : Sp_initial_cha_40,
        "Lower voltage cut-off [V]" : 1.2,
        "Upper voltage cut-off [V]" : 2.5,
        "Standard Potential L [V]" : EL0,
        "Standard Potential M [V]" : EM0,
        "Standard Potential H [V]" : EH0,
        "Exchange current density L [A.m-2]": il0,
        "Exchange current density M [A.m-2]": im0,
        "Exchange current density H [A.m-2]": ih0,
        "Active reaction area per cell [m2]":ar,
        "S saturation mass [g]": S_star_40,
        'Shuttle rate coefficient during charge [s-1]': ks_40,
        'Dissolution rate [s-1]': kd_40,
        'Reference Temperature [K]': T_40
        }
)

# Instantiate simulation object 
sim_40_cha1 = pybamm.Simulation(
    model_40,
    parameter_values = param_40,
    solver=pybamm.CasadiSolver(
        atol=1e-6,
        rtol=1e-6,
        dt_max = .01,
        extra_options_setup={"max_step_size": 0.1}, 
        max_step_decrease_count = 15,
    ),
)

# Run simulation
sim_40_cha1 = run_simulation(sim_40_cha1)


In [ ]:
# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

index_40 = len(sim_40_cha1.solution.t[sim_40_cha1.solution.t*0.0211/3600 < charge_data['40']['capacity'][-1] ])

capacity = abs(I)*sim_40_cha1.solution.t[:index_40]/3600
voltage = sim_40_cha1.solution['Terminal voltage [V]'].entries[:index_40]

plt.plot(charge_data['40']['capacity'],charge_data['40']['internal voltage'], linestyle='--')

plt.title("Charge Voltage Fit for $40^\circ$ Cell")
plt.plot(capacity, voltage)
plt.legend(['Experiment 0.1C', 'Simulation 0.1C'], loc = 'lower right', frameon=False)
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

In [ ]:
capacity_40 = .0211*sim_40_cha1.solution.t/3600
S4 = sim_40_cha1.solution['S4 [g]'].entries
S2 = sim_40_cha1.solution['S2 [g]'].entries
S1 = sim_40_cha1.solution['S [g]'].entries
Li_40 = S1 + 0.5*S2 + 0.25*S4 

filename = 'Cornish & Marinescu (2021)/res_data'
infile = open(filename,'rb')
res_data = pickle.load(infile)
infile.close()

res_discharge_data = res_data['discharge']
res_charge_data = res_data['charge']
quick_index = len(res_charge_data['40'][0][res_charge_data['40'][0]< .25])
plt.plot(res_charge_data['40'][0][:quick_index],res_charge_data['40'][1][:quick_index],linestyle='--')

max_Li_40 = max(Li_40)
max_R_40 = max(res_charge_data['40'][1])
gamma_40 = 1.2
alpha1_40 = (gamma_40-1)*max_Li_40*max_R_40
beta1_40 = gamma_40*max_Li_40
R_40 = alpha1_40/(beta1_40 - Li_40)

print(alpha1_40)
print(beta1_40)

plt.plot(capacity_40,R_40)

plt.title("Charge Resistance Fit for $40^\circ$ Cell")
plt.legend(['Experiment 0.1C', 'Simulation 0.1C'])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Resistance [$\Omega$]');

In [ ]:
loc = index_40
S1_initial_40 = sim_40_cha1.solution['S [g]'].entries[loc]
S2_initial_40 = sim_40_cha1.solution['S2 [g]'].entries[loc]
S4_initial_40 = sim_40_cha1.solution['S4 [g]'].entries[loc]
S8_initial_40 = sim_40_cha1.solution['S8 [g]'].entries[loc]
Sp_initial_40 = sim_40_cha1.solution['Precipitated Sulfur [g]'].entries[loc]
V_initial_40 = sim_40_cha1.solution['Terminal voltage [V]'].entries[loc]
print(S8_initial_40)
print(S4_initial_40)
print(S2_initial_40)
print(S1_initial_40)
print(Sp_initial_40)
print(V_initial_40)

In [ ]:
model_40_dis1 = models.Cornish2021()

# precipitation rate
kp_40 = 3

param_40_dis1 = model_40_dis1.default_parameter_values
param_40_dis1.update(
    {
        "Current function [A]": 2*.0211,
        "Initial Condition for Terminal Voltage [V]": V_initial_40,
        "Initial Condition for S8 ion [g]" : S8_initial_40,
        "Initial Condition for S4 ion [g]" : S4_initial_40,
        "Initial Condition for S2 ion [g]" : S2_initial_40,
        "Initial Condition for S ion [g]" : S1_initial_40,
        "Initial Condition for Precipitated Sulfur [g]" : Sp_initial_40,
        "Lower voltage cut-off [V]" : 1.2,
        "Upper voltage cut-off [V]" : 2.5,
        "Standard Potential L [V]" : EL0,
        "Standard Potential M [V]" : EM0,
        "Standard Potential H [V]" : EH0,
        "Exchange current density L [A.m-2]": il0,
        "Exchange current density M [A.m-2]": im0,
        "Exchange current density H [A.m-2]": ih0,
        "Active reaction area per cell [m2]":ar,
        "S saturation mass [g]": S_star_40,
        'Precipitation rate [s-1]': kp_40,
        'Reference Temperature [K]': T_40
        }
)


sim_40_dis1 = pybamm.Simulation(
    model_40_dis1,
    parameter_values = param_40_dis1,
    solver=pybamm.CasadiSolver(
        atol=1e-8,
        rtol=1e-6,
        dt_max = .01,
        extra_options_setup={"max_step_size": 0.01}, 
        max_step_decrease_count = 15,
    ),
)

sim_40_dis1 = run_simulation(sim_40_dis1, time_in_seconds = 19000)


In [ ]:
plt.plot(sim_40_dis1.solution.t*0.0422/3600, sim_40_dis1.solution['Terminal voltage [V]'].entries)

# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

plt.plot(discharge_data['40']['capacity'],discharge_data['40']['internal voltage'], linestyle='--')

plt.title("Disharge Voltage Fit for $40^\circ$ Cell")
plt.legend(["Simulation 0.2C","Experiment 0.2C" ])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

# 1.3 The 20 degree Cell

In [ ]:
# Experimental cell capacity
cap = 0.211

# Parameters taken from Marinescu et al. (2016)
Ms = 32
F = 9.649e4
R = 8.3145
T_20 = 303.15 - 10
ar = 0.960
nu = 0.0114

f_h = ((4 ** 2) * Ms * nu / 8)**(1/4)
f_m = ((2 ** 2) * Ms * nu / 4)**(1/2)
f_l = ((1 ** 2) * Ms * nu / 2)**(1/2)

# Current at 0.2C rate
I = -cap*0.1

# New standard potentials
EL0 = 1.9
EM0 = 2.41
EH0 = 2.43

# Exchange current constants
ih0 = 5
im0 = 5
il0 = 5

# Saturation mass constant
S_star_20 = 8e-7

# dissolution rate
kd_20 = 6500

# shuttle rate
ks_20 = 0.000001

# Initial value of S1 anion at start of charge
S1_initial_cha_20 = 5e-4

# Initial value of voltage at start of charge
V_initial_cha = 2.0

# Function for which root yields i_L = I
def S2_func(S1):
    constant = np.exp((2*F/(R*T_20))*(V_initial_cha - (2*R*T_20/F)*np.arcsinh(I/(-2*ar*il0)) - EL0 ))/(f_l**2)
    return constant*(S1**2)

# Function from equation (9b) in Cornish & Marinescu (2021)
def S4_func(S2):
    constant = np.exp((2*F/(R*T_20))*(V_initial_cha  - EM0 ))/(f_m**2)
    return constant*(S2**2)

# Function from equation (9a) in Cornish & Marinescu (2021)
def S8_func(S4):
    constant = np.exp((4*F/(R*T_20))*(V_initial_cha - EH0 ))/(f_h**4)
    return constant*(S4**2)

# Initial charge value of S2
S2_initial_cha_20 = S2_func(S1_initial_cha_20)

# Initial charge value of S2
S4_initial_cha_20 = S4_func(S2_initial_cha_20)

# Initial charge value of S2
S8_initial_cha_20 = S8_func(S4_initial_cha_20)

# Initial value of Precipitated sulfur at start of charge
# mass conservation across cells
Sp_initial_cha_20 = (S8_initial_cha_30 + 
                     S4_initial_cha_30 + 
                     S2_initial_cha_30 + 
                     S1_initial_cha_30 + 
                     Sp_initial_cha_30) - (S8_initial_cha_20 +
                                          S4_initial_cha_20 + 
                                          S2_initial_cha_20 + 
                                          S1_initial_cha_20)

# Initial high plateau potential 
EH_20 = EH0 + (R*T_20)*np.log(f_h*(S8_initial_cha_20**(1/4))/(S4_initial_cha_20**(1/2)))/(F)

# Initial high plateau over-potential
eta_H_20 = V_initial_cha - EH_20

# Initial high plateau current
iH_20 = -2*ih0*ar*np.sinh(F*eta_H_20/(2*R*T_20))

# Initial middle plateau potential
EM_20 = EM0 + (R*T_20)*np.log(f_m*(S4_initial_cha_20**(1/2))/S2_initial_cha_20)/(F)

# Initial middle plateau over-potentia
eta_M_20 = V_initial_cha - EM_20

# Initial middle plateau current
iM_20 = -2*im0*ar*np.sinh(F*eta_M_20/(2*R*T_20))

# Initial low plateau potential
EL_20 = EL0 + (R*T_20/F)*np.log(f_l*(S2_initial_cha_20**(1/2))/S1_initial_cha_20)

# Initial low plateau over-potentia
eta_L_20 = V_initial_cha - EL_20

# Initial low plateau current
iL_20 = -2*il0*ar*np.sinh(F*eta_L_20/(2*R*T_20))

print('The initial charge value of precipitated sulfur is {} g'.format(Sp_initial_cha_20))
print('The initial charge value of S1 anion is {} g'.format(S1_initial_cha_20))
print('The initial charge value of S2 anion is {} g'.format(S2_initial_cha_20))
print('The initial charge value of S4 anion is {} g'.format(S4_initial_cha_20))
print('The initial charge value of S8 anion is {} g'.format(S8_initial_cha_20))
print('The initial charge value of voltage is {} V'.format(V_initial_cha))
print('The initial charge value of the high plateau potential is {} V'.format(EH_20))
print('The initial charge value of the high plateau over-potential is {} V'.format(eta_H_20))
print('The initial charge value of the high plateau current is {} A'.format(iH_20))
print('The initial charge value of the middle plateau potential is {} V'.format(EM_20))
print('The initial charge value of the middle plateau over-potential is {} V'.format(eta_M_20))
print('The initial charge value of the middle plateau current is {} A'.format(iM_20))
print('The initial charge value of the low plateau potential is {} V'.format(EL_20))
print('The initial charge value of the low plateau over-potential is {} V'.format(eta_L_20))
print('The initial charge value of the low plateau current is {} A'.format(iL_20))



In [ ]:
# Instantiate the model
model_20 = models.Cornish2021()

# Obtain model parameter dictionary
param_20 = model_20.default_parameter_values

# Update the parameters for charge
param_20.update(
    {
        "Current function [A]": I,
        "Initial Condition for Terminal Voltage [V]": V_initial_cha,
        "Initial Condition for S8 ion [g]" : S8_initial_cha_20,
        "Initial Condition for S4 ion [g]" : S4_initial_cha_20,
        "Initial Condition for S2 ion [g]" : S2_initial_cha_20,
        "Initial Condition for S ion [g]" : S1_initial_cha_20,
        "Initial Condition for Precipitated Sulfur [g]" : Sp_initial_cha_20,
        "Lower voltage cut-off [V]" : 1.2,
        "Upper voltage cut-off [V]" : 2.5,
        "Standard Potential L [V]" : EL0,
        "Standard Potential M [V]" : EM0,
        "Standard Potential H [V]" : EH0,
        "Exchange current density L [A.m-2]": il0,
        "Exchange current density M [A.m-2]": im0,
        "Exchange current density H [A.m-2]": ih0,
        "Active reaction area per cell [m2]":ar,
        "S saturation mass [g]": S_star_20,
        'Shuttle rate coefficient during charge [s-1]': ks_20,
        'Dissolution rate [s-1]': kd_20,
        'Reference Temperature [K]': T_20
        }
)

# Instantiate simulation object 
sim_20_cha1 = pybamm.Simulation(
    model_20,
    parameter_values = param_20,
    solver=pybamm.CasadiSolver(
        atol=1e-6,
        rtol=1e-6,
        dt_max = .01,
        extra_options_setup={"max_step_size": 0.1}, 
        max_step_decrease_count = 15,
    ),
)

# run simulation
sim_20_cha1 = run_simulation(sim_20_cha1)



In [ ]:
# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

plt.plot(charge_data['20']['capacity'],charge_data['20']['internal voltage'], linestyle='--')

index_20 = len(sim_20_cha1.solution.t[sim_20_cha1.solution.t*0.0211/3600 < charge_data['20']['capacity'][-1]])

capacity = abs(I)*sim_20_cha1.solution.t[:index_20]/3600
voltage = sim_20_cha1.solution['Terminal voltage [V]'].entries[:index_20]


plt.title("Charge Voltage Fit for $20^\circ$ Cell")
plt.plot(capacity, voltage)
plt.legend(['Experiment 0.1C', 'Simulation 0.1C'], loc = 'lower right', frameon=False)
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

In [ ]:
capacity_20 = .0211*sim_20_cha1.solution.t/3600
S4 = sim_20_cha1.solution['S4 [g]'].entries
S2 = sim_20_cha1.solution['S2 [g]'].entries
S1 = sim_20_cha1.solution['S [g]'].entries
Li_20 = S1 + 0.5*S2 + 0.25*S4 

filename = 'Cornish & Marinescu (2021)/res_data'
infile = open(filename,'rb')
res_data = pickle.load(infile)
infile.close()

res_discharge_data = res_data['discharge']
res_charge_data = res_data['charge']
quick_index = len(res_charge_data['20'][0][res_charge_data['20'][0]< .25])
plt.plot(res_charge_data['20'][0][:quick_index],res_charge_data['20'][1][:quick_index],linestyle='--')

max_Li_20 = max(Li_20)
max_R_20 = max(res_charge_data['20'][1])
gamma_20 = 1.5
alpha1_20 = (gamma_20-1)*max_Li_20*max_R_20
beta1_20 = gamma_20*max_Li_20
R_20 = alpha1_20/(beta1_20 - Li_20)

print(alpha1_20)
print(beta1_20)

plt.plot(capacity_20,R_20)

plt.title("Charge Resistance Fit for $20^\circ$ Cell")
plt.legend(['Experiment 0.1C', 'Simulation 0.1C'])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Resistance [$\Omega$]');

In [ ]:
loc = index_20
S1_initial_20 = sim_20_cha1.solution['S [g]'].entries[loc]
S2_initial_20 = sim_20_cha1.solution['S2 [g]'].entries[loc]
S4_initial_20 = sim_20_cha1.solution['S4 [g]'].entries[loc]
S8_initial_20 = sim_20_cha1.solution['S8 [g]'].entries[loc]
Sp_initial_20 = sim_20_cha1.solution['Precipitated Sulfur [g]'].entries[loc]
V_initial_20 = sim_20_cha1.solution['Terminal voltage [V]'].entries[loc]
print(S8_initial_20)
print(S4_initial_20)
print(S2_initial_20)
print(S1_initial_20)
print(Sp_initial_20)
print(V_initial_20)

In [ ]:
model_20_dis1 = models.Cornish2021()

# Precipitation rate
kp_20

param_20_dis1 = model_20_dis1.default_parameter_values
param_20_dis1.update(
    {
        "Current function [A]": 2*.0211,
        "Initial Condition for Terminal Voltage [V]": V_initial_20,
        "Initial Condition for S8 ion [g]" : S8_initial_20,
        "Initial Condition for S4 ion [g]" : S4_initial_20,
        "Initial Condition for S2 ion [g]" : S2_initial_20,
        "Initial Condition for S ion [g]" : S1_initial_20,
        "Initial Condition for Precipitated Sulfur [g]" : Sp_initial_20,
        "Lower voltage cut-off [V]" : 1.2,
        "Upper voltage cut-off [V]" : 2.5,
        "Standard Potential L [V]" : EL0,
        "Standard Potential M [V]" : EM0,
        "Standard Potential H [V]" : EH0,
        "Exchange current density L [A.m-2]": il0,
        "Exchange current density M [A.m-2]": im0,
        "Exchange current density H [A.m-2]": ih0,
        "Active reaction area per cell [m2]":ar,
        "S saturation mass [g]": S_star_20,
        'Precipitation rate [s-1]': kp_20,
        'Reference Temperature [K]': T_20
        }
)


sim_20_dis1 = pybamm.Simulation(
    model_20_dis1,
    parameter_values = param_20_dis1,
    solver=pybamm.CasadiSolver(
        atol=1e-8,
        rtol=1e-6,
        dt_max = .01,
        extra_options_setup={"max_step_size": 0.01}, 
        max_step_decrease_count = 15,
    ),
)

sim_20_dis1 = run_simulation(sim_20_dis1, time_in_seconds = 15920)#18130)



In [ ]:
plt.plot(sim_20_dis1.solution.t*0.0422/3600, sim_20_dis1.solution['Terminal voltage [V]'].entries)

# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

plt.plot(discharge_data['20']['capacity'],discharge_data['20']['internal voltage'], linestyle='--')

plt.title("Disharge Voltage Fit for $20^\circ$ Cell")
plt.legend(["sim 0.2C","exp 0.2C" ])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

# 2.0 All Fitted Results

In [ ]:
index_20 = len(sim_20_cha1.solution.t[sim_20_cha1.solution.t < 36001])
capacity = abs(I)*sim_20_cha1.solution.t[:index_20]/3600
voltage = sim_20_cha1.solution['Terminal voltage [V]'].entries[:index_20]
plt.plot(capacity, voltage, color = 'blue')

index_30 = len(sim_30_cha1.solution.t[sim_30_cha1.solution.t < 36001])
capacity = abs(I)*sim_30_cha1.solution.t[:index_30]/3600
voltage = sim_30_cha1.solution['Terminal voltage [V]'].entries[:index_30]
plt.plot(capacity, voltage, color = 'green')

index_40 = len(sim_40_cha1.solution.t[sim_40_cha1.solution.t < 36001])
capacity = abs(I)*sim_40_cha1.solution.t[:index_40]/3600
voltage = sim_40_cha1.solution['Terminal voltage [V]'].entries[:index_40]
plt.plot(capacity, voltage, color = 'red')

# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

plt.plot(charge_data['20']['capacity'],charge_data['20']['internal voltage'], linestyle='--', color = 'blue')
plt.plot(charge_data['30']['capacity'],charge_data['30']['internal voltage'], linestyle='--', color = 'green')
plt.plot(charge_data['40']['capacity'],charge_data['40']['internal voltage'], linestyle='--', color = 'red')


plt.legend(["Simulation $20^\circ$C",
            "Simulation $30^\circ$C",
            "Simulation $40^\circ$C",
            "Experiment $20^\circ$C",
            "Experiment $30^\circ$C",
            "Experiment $40^\circ$C",
            ])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]')

In [ ]:
plt.plot(capacity_20,R_20, color = 'blue')
plt.plot(capacity_30,R_30, color = 'green')
plt.plot(capacity_40,R_40, color = 'red')

filename = 'Cornish & Marinescu (2021)/res_data'
infile = open(filename,'rb')
res_data = pickle.load(infile)
infile.close()

res_discharge_data = res_data['discharge']
res_charge_data = res_data['charge']

quick_index = len(res_charge_data['20'][0][res_charge_data['20'][0]< .25])
plt.plot(res_charge_data['20'][0][:quick_index],res_charge_data['20'][1][:quick_index],linestyle='--', color = 'blue')

quick_index = len(res_charge_data['30'][0][res_charge_data['30'][0]< .25])
plt.plot(res_charge_data['30'][0][:quick_index],res_charge_data['30'][1][:quick_index],linestyle='--', color = 'green')

quick_index = len(res_charge_data['40'][0][res_charge_data['40'][0]< .25])
plt.plot(res_charge_data['40'][0][:quick_index],res_charge_data['40'][1][:quick_index],linestyle='--', color = 'red')

plt.legend(["Simulation $20^\circ$C",
            "Simulation $30^\circ$C",
            "Simulation $40^\circ$C",
            "Experiment $20^\circ$C",
            "Experiment $30^\circ$C",
            "Experiment $40^\circ$C",
            ])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Resistance [$\Omega$]');

In [ ]:
plt.plot(sim_20_dis1.solution.t*0.0422/3600, sim_20_dis1.solution['Terminal voltage [V]'].entries, color = 'blue')
plt.plot(sim_30_dis1.solution.t*0.0422/3600, sim_30_dis1.solution['Terminal voltage [V]'].entries, color = 'green')
plt.plot(sim_40_dis1.solution.t*0.0422/3600, sim_40_dis1.solution['Terminal voltage [V]'].entries, color = 'red')

# get experimental data
filename = 'Cornish & Marinescu (2021)/vol_data'
infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

discharge_data = data_dict['discharge']
charge_data = data_dict['charge']

plt.plot(discharge_data['20']['capacity'],discharge_data['20']['internal voltage'], linestyle='--', color = 'blue')
plt.plot(discharge_data['30']['capacity'],discharge_data['30']['internal voltage'], linestyle='--', color = 'green')
plt.plot(discharge_data['40']['capacity'],discharge_data['40']['internal voltage'], linestyle='--', color = 'red')


plt.legend(["Simulation $20^\circ$C",
            "Simulation $30^\circ$C",
            "Simulation $40^\circ$C",
            "Experiment $20^\circ$C",
            "Experiment $30^\circ$C",
            "Experiment $40^\circ$C",
            ])
plt.xlabel('Capacity [Ah]')
plt.ylabel('Voltage [V]');

In [ ]:
print(kd_20)
print(kd_30)
print(kd_40)

print(S_star_20)
print(S_star_30)
print(S_star_40)

In [ ]:
capacity_dis_20 = 2*.0211*sim_20_dis1.solution.t/3600
S4 = sim_20_dis1.solution['S4 [g]'].entries
S2 = sim_20_dis1.solution['S2 [g]'].entries
S1 = sim_20_dis1.solution['S [g]'].entries
Li_dis_20 = S1 + 0.5*S2 + 0.25*S4 
R_dis_20 = alpha1_20/(beta1_20 - Li_dis_20)
plt.plot(capacity_dis_20, R_dis_20, color = 'blue')

capacity_dis_30 = 2*.0211*sim_30_dis1.solution.t/3600
S4 = sim_30_dis1.solution['S4 [g]'].entries
S2 = sim_30_dis1.solution['S2 [g]'].entries
S1 = sim_30_dis1.solution['S [g]'].entries
Li_dis_30 = S1 + 0.5*S2 + 0.25*S4 
R_dis_30 = alpha_30/(beta_30 - Li_dis_30)
plt.plot(capacity_dis_30, R_dis_30, color = 'green')

capacity_dis_40 = 2*.0211*sim_40_dis1.solution.t/3600
S4 = sim_40_dis1.solution['S4 [g]'].entries
S2 = sim_40_dis1.solution['S2 [g]'].entries
S1 = sim_40_dis1.solution['S [g]'].entries
Li_dis_40 = S1 + 0.5*S2 + 0.25*S4 
R_dis_40 = alpha1_40/(beta1_40 - Li_dis_40)
plt.plot(capacity_dis_40, R_dis_40, color = 'red')

filename = 'Cornish & Marinescu (2021)/res_data'
infile = open(filename,'rb')
res_data = pickle.load(infile)
infile.close()

res_discharge_data = res_data['discharge']
res_charge_data = res_data['charge']

plt.plot(res_discharge_data['20'][0],res_discharge_data['20'][1],linestyle='--',color = 'blue')
plt.plot(res_discharge_data['30'][0],res_discharge_data['30'][1],linestyle='--', color = 'green')
plt.plot(res_discharge_data['40'][0],res_discharge_data['40'][1],linestyle='--', color = 'red')
